In [451]:
import pandas as pd

### Read Data File

In [452]:
df = pd.read_excel("Eksamen-2024/2024-json ANONYM.xlsx", engine = 'openpyxl')

In [453]:
df1 = pd.read_excel("Eksamen-2024/2023-json-ANONYM.xlsx", engine = 'openpyxl')

#### Finding Missing Data

In [454]:
# finding missing data/Null Data
def find_missing_data(df):
    """Check for missing values and return boolean"""
    df.isna() # return boolean True if values are Null and False if not missing. 
    return df

### Check for duplicated values

In [455]:
def find_duplicated(df):
    """Check for duplicates and return boolean."""
    
    df.duplicated()
    return df

### Drop Duplicates

In [456]:
# dropping duplicates
def drop_duplicated(df):
    """Drop duplicated rows"""
    
    df.drop_duplicates(inplace = True)
    return df

### Fill Null Values

In [457]:
def fill_missing_values(df):
    """Fill in missing values in specified columns"""
    
    cols_to_fill = [
        'Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_inspera_',
        'Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_insper.2',
        'Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_insper.1'
    ]
    
    for col in cols_to_fill:
        if col in df.columns:
            df[col] = df[col].fillna("Unknown")
    
    return df

### Drop missing/null values 

In [458]:
# dropping missing/null values
def drop_missing(df):
    """Drop missing (null) values"""
    df.dropna(inplace = True)
    # rest default indexes after dropping so it not longer shows older indexes numbers
    df = df.reset_index(drop=True)
    return df

### Drop Non-Useful Columns by Name

In [459]:
# dropping columns using their indexes. 
#df.drop(df.columns[[0, 1, 2, 9, 10, 11, 12, 13, 16, 17, 20, 23, 25, 26]], axis = 1, inplace = True)

def drop_columns(df):
    """ Drop non-useful columns """
    
    # defining columns to drop based on the pattern found on the JSON file
    columns_to_drop = ["Column1.result.sourcedId", 
         "Column1.result.ext_inspera_userAssessmentSetupId", 
         "Column1.result.ext_inspera_userAssessmentId",
         "Column1.result.ext_inspera_attendance", 
         "Column1.result.lineItem.sourcedId",
         "Column1.result.lineItem.type", 
         "Column1.result.student.sourcedId",
         "Column1.result.student.type", 
         "Column1.result.ext_inspera_questions.ext_inspera_questionId",
         "Column1.result.ext_inspera_questions.ext_inspera_questionContentItemId", 
         "Column1.result.ext_inspera_questions.ext_inspera_questionWeight",
         "Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_inspera_",
         "Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_insper.2",
         "Column1.result.score", "Autoscore", "Column1.result.dateLastModified" ]

    # condition to take only columns matching the current dataframe (df) 
    current_df_columns = [col for col in columns_to_drop if col in df.columns]

    # drop these columns
    df.drop(current_df_columns, axis=1, inplace=True)
        
    return df

### Renaming Columns 

In [460]:
def rename_columns(df):
    """Rename columns to be easily readable"""
    
    rename_map = {
        "Column1.result.ext_inspera_startTime" : "Starttid",
        "Column1.result.ext_inspera_endTime" : "Sluttid",
        "Column1.result.ext_inspera_extraTimeMins" : "Ekstratid (minutter)",
        "Column1.result.ext_inspera_incidentTimeMins" : "Hendelsestid (minutter)",
        "Column1Column1.result.ext_inspera_candidateId" : "Kandidat‑ID",
        "Column1.result.ext_inspera_autoScore" : "Resultatpoeng",
        "Column1.result.ext_inspera_questions.ext_inspera_maxQuestionScore" : "Oppgave MaxPoeng",
        "Column1.result.ext_inspera_questions.ext_inspera_questionNumber" : "Oppgave",
        "Column1.result.ext_inspera_questions.ext_inspera_questionTitle" : "Oppgavetittel",
        "Column1.result.ext_inspera_questions.ext_inspera_durationSeconds" : "Oppgavetid (sekunder)",
        "Column1.result.ext_inspera_questions.ext_inspera_autoScore" : "Oppnådd poeng per oppgave",
        "Column1.result.ext_inspera_questions.ext_inspera_candidateResponses.ext_insper.1" : "Kandidatens svar",
        "Column1" : "Kandidat‑ID",
        "Oppgave Tid" : "Oppgavetid (sekunder)",
        "Oppgave Poeng" : "Oppnådd poeng per oppgave"     
    }

    filtered_rename_map = {k: v for k, v in rename_map.items() if k in df.columns}

    df = df.rename(columns = filtered_rename_map)
    return df
    

## Convert Date into time 

In [461]:
def convert_into_datetime(df):
    """ Convert dates into datetime format and extract time only for start and end time"""
    # convert specified columns into datetime format
    # df['Sist endret'] = pd.to_datetime(df['Sist endret']).dt.tz_localize(None) # dt.tz_localize(None) remove +00:00 UTC time information

    # convert to datetime
    df['Starttid_full'] = pd.to_datetime(df['Starttid'])
    df['Sluttid_full']  = pd.to_datetime(df['Sluttid'])

    # convert to time format accepted by excel
    df['Starttid'] = df['Starttid_full'].dt.strftime('%H:%M:%S')
    df['Sluttid']  = df['Sluttid_full'].dt.strftime('%H:%M:%S')

    return df 

### Adding column for time related statistics 

In [462]:
def add_time_usage_columns(df, max_minutes = 240):
    """
    Adds time usage columns to the DataFrame (df):
      - Brukt_tid: total time used in minutes
      - Tid_igjen: remaining time in minutes
      - Prosent_brukt: percent of allowed time used
    """
    df = df.copy() 
    df["Brukt tid (minutter)"] = round((df['Sluttid_full'] - df['Starttid_full']).dt.total_seconds() / 60 ,2)
    df["Tid igjen (minutter)"] = round(max_minutes - df["Brukt tid (minutter)"], 2)
    df["%Tid brukt"] = round((df["Brukt tid (minutter)"] / max_minutes) * 100, 2)
    return df

### Re-order Columns

In [463]:
def reorder_columns(df):
    """Re-order columns in the dataframe"""

    # intilizing desired order 
    ordered_columns = [
    "Kandidat‑ID",
    "Oppgave",
    "Oppgavetittel",
    "Starttid",
    "Sluttid",
    "Ekstratid (minutter)",
    "Hendelsestid (minutter)",
    "Oppgavetid (sekunder)",
    "Kandidatens svar",
    "Oppnådd poeng per oppgav",
    "Oppgave MaxPoeng",
    "Brukt tid (minutter)",
    "Tid igjen (minutter)",
    "%Tid brukt",
    "Resultatpoeng"]

    # check if columns name exist in current Dataframe (df)
    current_df_columns = [col for col in ordered_columns if col in df.columns]

    # re-order df based on the new order
    df = df[current_df_columns]
    
    return df

## Cleaning 2023 Exam File

In [464]:
# Dropping duplicated values 
df1 = drop_duplicated(df1)
# df1

In [465]:
# fill missing value with unknown
df1 = fill_missing_values(df1)
# df1 

In [466]:
# Dropping missing values 
# df1 = drop_missing(df1)
# df1

In [467]:
# Dropping non-useful columns
df1 = drop_columns(df1)
# df1

In [468]:
# Renaming Columns
df1 = rename_columns(df1)
# df1

In [469]:
# changing time and date format
df1 = convert_into_datetime(df1)
# df1

In [482]:
# add time usage columns
df1 = add_time_usage_columns(df1, 240)
# df1

KeyError: 'Sluttid_full'

In [471]:
# re-order columns 
df1 = reorder_columns(df1)
# df1

,Kandidat‑ID,Oppgave,Oppgavetittel,Starttid,Sluttid,Ekstratid (minutter),Hendelsestid (minutter),Oppgavetid (sekunder),Oppgave MaxPoeng,Brukt tid (minutter),Tid igjen (minutter),%Tid brukt,Resultatpoeng
0,13747,1.1,Dataformater,08:00:12,11:54:57,0,0,641,4.5,234.75,5.25,97.81,63.0
1,13747,1.1,Dataformater,08:00:12,11:54:57,0,0,641,4.5,234.75,5.25,97.81,63.0
2,13747,1.1,Dataformater,08:00:12,11:54:57,0,0,641,4.5,234.75,5.25,97.81,63.0
3,13747,1.1,Dataformater,08:00:12,11:54:57,0,0,641,4.5,234.75,5.25,97.81,63.0
4,13747,1.1,Dataformater,08:00:12,11:54:57,0,0,641,4.5,234.75,5.25,97.81,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64804,13624,4.8,Transportprotokoller,08:00:26,11:59:35,0,0,654,5.0,239.15,0.85,99.65,77.5
64805,13624,4.8,Transportprotokoller,08:00:26,11:59:35,0,0,654,5.0,239.15,0.85,99.65,77.5
64806,13624,4.9,Content Delivery Network,08:00:26,11:59:35,0,0,158,3.0,239.15,0.85,99.65,77.5
64807,13624,4.9,Content Delivery Network,08:00:26,11:59:35,0,0,158,3.0,239.15,0.85,99.65,77.5


## Cleaning 2024 Exam File

In [472]:
# Dropping duplicated values 
df = drop_duplicated(df)

In [473]:
# Fill missing value with unknown
df = fill_missing_values(df)
# df

In [474]:
# Dropping missing values 
# df = drop_missing(df)
# df

In [475]:
# Dropping non-useful columns
df = drop_columns(df)

In [476]:
# Renaming Columns
df = rename_columns(df)
# df

In [477]:
# changing time and date format
df = convert_into_datetime(df)

In [478]:
# add time usage columns
df = add_time_usage_columns(df, 240)
df

,Starttid,Sluttid,Ekstratid (minutter),Hendelsestid (minutter),Kandidat‑ID,Resultatpoeng,Oppgave MaxPoeng,Oppgave,Oppgavetittel,Oppgavetid (sekunder),Oppnådd poeng per oppgave,Kandidatens svar,Starttid_full,Sluttid_full,Brukt tid (minutter),Tid igjen (minutter),%Tid brukt
0,14:00:02,15:57:52,0,0,17107,71.61,2.4,1.1,Tallsystemer,85,2.4,Heksadesimale tall,2024-12-11 14:00:02+00:00,2024-12-11 15:57:52+00:00,117.83,122.17,49.10
1,14:00:02,15:57:52,0,0,17107,71.61,2.4,1.1,Tallsystemer,85,2.4,Oktale tall,2024-12-11 14:00:02+00:00,2024-12-11 15:57:52+00:00,117.83,122.17,49.10
2,14:00:02,15:57:52,0,0,17107,71.61,2.4,1.1,Tallsystemer,85,2.4,Titallsystemet,2024-12-11 14:00:02+00:00,2024-12-11 15:57:52+00:00,117.83,122.17,49.10
3,14:00:02,15:57:52,0,0,17107,71.61,2.4,1.1,Tallsystemer,85,2.4,Heksadesimale tall,2024-12-11 14:00:02+00:00,2024-12-11 15:57:52+00:00,117.83,122.17,49.10
4,14:00:02,15:57:52,0,0,17107,71.61,2.4,1.1,Tallsystemer,85,2.4,Titallsystemet,2024-12-11 14:00:02+00:00,2024-12-11 15:57:52+00:00,117.83,122.17,49.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65875,14:00:15,16:38:05,0,0,17383,64.09,2.0,4.6,Transportlagsprotokoller,335,1.0,Unknown,2024-12-11 14:00:15+00:00,2024-12-11 16:38:05+00:00,157.83,82.17,65.76
65876,14:00:15,16:38:05,0,0,17383,64.09,1.0,4.7,HTTP,348,0.0,HTTP-forespørsler kan ikke multiplekses over d...,2024-12-11 14:00:15+00:00,2024-12-11 16:38:05+00:00,157.83,82.17,65.76
65877,14:00:15,16:38:05,0,0,17383,64.09,1.0,4.7,HTTP,348,0.0,Den samme TCP-forbindelsen blir gjenbrukt til ...,2024-12-11 14:00:15+00:00,2024-12-11 16:38:05+00:00,157.83,82.17,65.76
65878,14:00:15,16:38:05,0,0,17383,64.09,2.0,4.8,HTTP-streaming,200,0.0,Video is divided in small segments and differe...,2024-12-11 14:00:15+00:00,2024-12-11 16:38:05+00:00,157.83,82.17,65.76


In [479]:
# re-order columns
df = reorder_columns(df)
df.tail()

,Kandidat‑ID,Oppgave,Oppgavetittel,Starttid,Sluttid,Ekstratid (minutter),Hendelsestid (minutter),Oppgavetid (sekunder),Kandidatens svar,Oppgave MaxPoeng,Brukt tid (minutter),Tid igjen (minutter),%Tid brukt,Resultatpoeng
65875,17383,4.6,Transportlagsprotokoller,14:00:15,16:38:05,0,0,335,Unknown,2.0,157.83,82.17,65.76,64.09
65876,17383,4.7,HTTP,14:00:15,16:38:05,0,0,348,HTTP-forespørsler kan ikke multiplekses over d...,1.0,157.83,82.17,65.76,64.09
65877,17383,4.7,HTTP,14:00:15,16:38:05,0,0,348,Den samme TCP-forbindelsen blir gjenbrukt til ...,1.0,157.83,82.17,65.76,64.09
65878,17383,4.8,HTTP-streaming,14:00:15,16:38:05,0,0,200,Video is divided in small segments and differe...,2.0,157.83,82.17,65.76,64.09
65879,17383,4.8,HTTP-streaming,14:00:15,16:38:05,0,0,200,Streaming over HTTP only uses UDP to transfer ...,2.0,157.83,82.17,65.76,64.09


### Export Cleaned data to excel

In [480]:
# df1.to_excel('clean_exam_2023.xlsx', index = False)

In [481]:
# df.to_excel('clean_exam_2024.xlsx', index = False)